In [29]:
import sys
sys.path.append('..')

In [30]:
import pandas as pd
import numpy as np
import re
from functools import partial
from src.features_extraction import *
from src.targets_extraction import *
import matplotlib.pyplot as plt
from src.useful_tools import *
from sklearn.model_selection import ValidationCurveDisplay

In [31]:
import talib as ta
import sklearn.tree as tree
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

In [36]:
df = pd.read_csv("../data/data.csv")
df = prepare_desired_pos(df, lag=10, multiplier=10)

Generating desired position...
Desired position generated


In [37]:
df = generate_all_features_df(df, [i for i in range(7, 150, 7)])

Generating original features...
Generating momentum features...
Generating math features...
Generating pattern features...
Generating time features...
All features generated


In [38]:
df = keep_essentials(df)

,ADOSC_105,ADOSC_112,ADOSC_119,ADOSC_126,ADOSC_133,ADOSC_14,ADOSC_140,ADOSC_147,ADOSC_21,ADOSC_28,...,WILLR_91,WILLR_98,desired_pos_change,desired_pos_rolling,net_pos_signal,pos_change_signal,time_day_of_month,time_day_of_week,time_hour,time_minute
441,-22065.848939,-23257.642988,-24347.271586,-25340.487482,-26243.196209,8.784841,-27061.294969,-27800.560887,-955.818957,-2516.936395,...,-58.974000,-49.137078,26.0,28,long hold,strong buy,28,2,10,51
442,-21995.129916,-23192.507991,-24287.870779,-25286.875706,-26195.357242,-95.829346,-27019.159663,-27764.020899,-991.673038,-2505.875209,...,-62.289155,-51.899262,32.0,60,long hold,strong buy,28,2,10,52
443,-21923.557760,-23126.469418,-24227.524862,-25232.285249,-26146.513036,-181.683678,-26975.998682,-27726.440110,-1020.431823,-2491.752849,...,-59.199813,-49.325226,25.0,85,long hold,strong buy,28,2,10,53
444,-21848.154236,-23056.733079,-24163.602186,-25174.229058,-26094.306004,-230.673924,-26929.571031,-27685.683074,-1028.547443,-2463.969866,...,-52.555661,-50.744839,18.0,103,long hold,strong buy,28,2,10,54
445,-21777.147871,-22991.015028,-24103.366342,-25119.568308,-26045.237084,-305.146101,-26886.053289,-27647.632788,-1057.110903,-2453.249117,...,-49.452311,-47.748415,16.0,119,long hold,strong buy,28,2,10,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379710,14757.566022,16743.186337,18746.339615,20754.601943,22757.407101,-399.803190,24745.884965,26712.680226,-909.346163,-808.933483,...,-80.383770,-85.718050,8.0,-36,short hold,buy,27,3,22,45
379711,14670.719791,16652.006847,18651.549525,20656.860979,22657.318147,-294.803166,24643.998474,26609.497750,-815.902381,-744.680358,...,-78.367822,-82.316806,5.0,-31,short hold,buy,27,3,22,46
379712,14592.218407,16568.653391,18564.120974,20566.063207,22563.793558,-151.375217,24548.331637,26512.219884,-693.612536,-656.362476,...,-71.103372,-73.736628,-2.0,-33,short hold,sell,27,3,22,47
379713,14519.034181,16490.297443,18481.400158,20479.709318,22474.471604,3.338113,24456.646009,26418.719286,-559.720414,-556.556868,...,-72.034022,-74.582471,1.0,-32,short hold,meh,27,3,22,48


In [40]:
df = drop_ohlcv_cols(df)

In [41]:
X, y = split_features_target(df)

Splitting features/target


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
clf = tree.DecisionTreeClassifier(min_samples_split=10000, min_samples_leaf=1000)

In [43]:
clf.fit(X_train, y_train['pos_change_signal'])

DecisionTreeClassifier(min_samples_leaf=1000, min_samples_split=10000)

In [45]:
ValidationCurveDisplay.from_estimator(
    clf, X, y['pos_change_signal'], param_name='max_depth', param_range=[i for i in range(1, 50, 5)], scoring='accuracy'
)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, min_samples_split=10000)

In [ ]:
X_train

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print(metrics.classification_report(y_test, clf.predict(X_test)))

In [ ]:
df = pd.read_csv("../data/data.csv")

In [ ]:
df = prepare_desired_pos(df, 10, 10)

In [ ]:
df.desired_pos_change.plot()

In [ ]:
df[['pos_change_signal', 'net_pos_signal']].value_counts().plot(kind='pie')

In [ ]:
rle(df, True)

In [ ]:
df['daily_pnl'] = df['desired_pos_rolling'] * (df['close'] - df['open'])

In [ ]:
df.daily_pnl.cumsum().plot()

In [ ]:
output = ta_map["SMA"](inputs)

In [ ]:
output

In [ ]:
ta.RSI.parameters

In [ ]:
from talib import abstract

abstract.SMA.parameters

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
generate_time_features(df)

In [ ]:
df.datetime

In [ ]:
with open("feature_names.txt", "r") as f:
    features = f.read()

In [ ]:
import re

# Save this for production use!

In [ ]:
with open("feature_names.txt", "r") as f:
    features = f.read()
feats = re.findall(r"(\w+\D)(\d+\b)", features)
func_pool = []

In [ ]:
for func in feats:
    if func[0] == "MACDHIST" or func[0] == "MACDSIGNAL":
        continue
    if func[0] == "MACDSIGNALFIX" or func[0] == "MACDHISTFIX":
        continue
    if func[0] == "STOCHRSI_k" or func[0] == "STOCHRSI_d":
        continue
    if func[0] == "HT_PHASORinphase" or func[0] == "HT_PHASORquadrature":
        func[0] = "HT_PHASOR"
    if func[0] == "HT_SINEsine" or func[0] == "HT_SINEleadsine":
        func[0] = "HT_SINE"
    function = getattr(ta, func[0])
    func_pool.append((function, int(func[1])))

In [ ]:
inspect.signature(func_pool[0][0])

In [ ]:
import inspect

In [ ]:
for func, lag in func_pool:
    parameters = inspect.signature(func).parameters

In [ ]:
import talib as ta

In [ ]:
getattr(ta, feats[0][0])